# Part1: Deployment and Data Upload

In [1]:
!ls

p4a.ipynb  p4b.ipynb


In [2]:
!pwd

/nb


In [2]:
#q1
!hdfs dfsadmin -fs hdfs://boss:9000/ -report

Configured Capacity: 51642105856 (48.10 GB)
Present Capacity: 23415353344 (21.81 GB)
DFS Remaining: 23415300096 (21.81 GB)
DFS Used: 53248 (52 KB)
DFS Used%: 0.00%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (2):

Name: 172.20.0.3:9866 (project-4-twhero-dn-2.project-4-twhero_default)
Hostname: 1a2d05ddcf1d
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 28672 (28 KB)
Non DFS Used: 14096596992 (13.13 GB)
DFS Remaining: 11707650048 (10.90 GB)
DFS Used%: 0.00%
DFS Remaining%: 45.34%
Config

In [6]:
!wget https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.csv

--2023-10-21 00:36:06--  https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.csv
Resolving pages.cs.wisc.edu (pages.cs.wisc.edu)... 128.105.7.9
Connecting to pages.cs.wisc.edu (pages.cs.wisc.edu)|128.105.7.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 174944099 (167M) [text/csv]
Saving to: ‘hdma-wi-2021.csv’

hdma-wi-2021.csv    100%[===================>] 166.84M  7.80MB/s    in 20s     

2023-10-21 00:36:26 (8.53 MB/s) - ‘hdma-wi-2021.csv’ saved [174944099/174944099]



In [6]:
!hdfs dfs -rm -f hdfs://boss:9000/single.csv
!hdfs dfs -rm -f hdfs://boss:9000/double.csv

Deleted hdfs://boss:9000/single.csv
Deleted hdfs://boss:9000/double.csv


In [7]:
!hdfs dfs -D dfs.replication=1 -D dfs.block.size=1048576 -cp ./hdma-wi-2021.csv hdfs://boss:9000/single.csv
!hdfs dfs -D dfs.replication=2 -D dfs.block.size=1048576 -cp ./hdma-wi-2021.csv hdfs://boss:9000/double.csv

In [4]:
#q2
!hdfs dfs -du -h hdfs://boss:9000/

166.8 M  333.7 M  hdfs://boss:9000/double.csv
166.8 M  166.8 M  hdfs://boss:9000/single.csv


# Part 2: WebHDFS

In [9]:
#q3
import requests
r = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=GETFILESTATUS")
r.json()

{'FileStatus': {'accessTime': 1698006412933,
  'blockSize': 1048576,
  'childrenNum': 0,
  'fileId': 16388,
  'group': 'supergroup',
  'length': 174944099,
  'modificationTime': 1698006417265,
  'owner': 'root',
  'pathSuffix': '',
  'permission': '644',
  'replication': 1,
  'storagePolicy': 0,
  'type': 'FILE'}}

In [13]:
#q4
!curl "http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset=0&noredirect=true"

{"Location":"http://1dfd0e6f5094:9864/webhdfs/v1/single.csv?op=OPEN&namenoderpcaddress=boss:9000&offset=0"}

In [19]:
#q5
import math
distrib = {}
offset = 0
length = 1048576
r = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=GETFILESTATUS")
myrange = math.ceil(r.json()['FileStatus']['length'] / r.json()['FileStatus']['blockSize'])
for i in range(myrange):
    r = requests.get(f"http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset={offset}&length={length}&noredirect=true")
    id = r.json()['Location'].split('/')[2]
    distrib[id] = distrib.get(id,0) + 1
    offset += length
distrib

{'1dfd0e6f5094:9864': 87, '5d7c816ef0dc:9864': 80}

# Part 3: PyArrow

In [9]:
import pyarrow as pa
import pyarrow.fs

In [10]:
#q6
hdfs = pa.fs.HadoopFileSystem('boss', 9000)
tmp = None
with hdfs.open_input_file('/single.csv') as f:
    tmp = f.read_at(10,0)
tmp

2023-10-23 20:12:55,221 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


b'activity_y'

In [22]:
#q7
import io
mycount = 0
with hdfs.open_input_file('/single.csv') as f:
    reader = io.TextIOWrapper(io.BufferedReader(f))
    for i,line in enumerate(reader):
        if 'Single Family' in line:
            mycount += 1
mycount
                                                           

444874